# Handle large data sets with BigQuery 
Tutorial created by [Thomas Belhalfaoui](https://www.linkedin.com/in/belhalfaoui/?originalSubdomain=fr) for [Ironhack](https://www.ironhack.com/).

## 1. Why and when to use big data engines?

Before we start, let's talk a bit about what big data engines are, why end when they are useful, and what are their upsides and downsides.

### 1.1. Relational databases and vertical scalability

When you have a dataset, you can just store it in a relational database (e.g. MySQL). If the dataset is larger, the solution is easy: just buy a bigger machine for your database (more RAM and larger hard drive)! This is what we call **vertical scalability**.

It works, but having to change the machine every time the dataset gets too big is not very handy.

Plus, but what happens when you reach dozens of terabytes of data? Most probably, you will not find any computer that has such a large hard drive, not even speaking of the RAM. And even for smaller datasets (a couple of terabytes), it may happen that some features of the MySQL database (mostly joins) will become almost unusable.

### 1.2. Big data engines, sharding and horizontal scalability

To solve this issues, there is an idea: why not **split** the data across multiple machines? This exists: it is called **sharding** (each **split** is a **shard**) and it is part of what we call **horizontal scalability**. This is what **big data storage and query engine** do.

But now that the data is spread across several machines, how do we know where each data lies? This is why the **sharding key** is for. For instance, if the sharding key can be a `year` column, then all rows with the same value of `year` will go the the same shard (machine). 

### 1.3. Big data engines and denormalization: loss of flexibility and data redundancy

So big data engines are perfect? Of course not. There is one big drawback: **we cannot (easily) do _joins_**.

Indeed, when two tables are sharded, making a _join_ between the two of them means moving a lot of data across the Internet. Imagine the data is sharded by `year` but you join on the `client_id` column. Most probably, for a given client ID, the rows of table 1 do not lie on the same machine as the rows of table 2. So one of the two has to travel on the Internet to go to the same machine for later aggregation.

And network transfer is _very_ slow (several orders of magnitude slower than reading from a hard drive). What is worse, the different shards do not have to be physically close - actually they can even be quite far away, sometimes in different countries.

So what do you do when you cannot do joins: you **pre-join**. This is called **denormalizing** the data. Instead of having multiple tables that relate to each other like in a relational (e.g. MySQL) database (aka a **normalized** format), we have one big table with many columns, with everything already joined together in some way.

Of course, you understand what the biggest drawback of this approach is: you have to decide in advance how you want to join (denormalize) the dataset and you can do it only once! You can do it several times but this means you store several copies of the full denormalized dataset. So you **loose the flexibility** that you had with relational databases.

And the second drawback is that it uses **much more storage space** since there is a lot of **data redundancy**. Imagine you have:
* A `Product` table with a `productId` column and all product metadata (`productName`, etc.),
* A `Sales` tables with `date` and `productId`.

Now we denormalize the dataset (aka join on `productId`). So each time a product is sold, we don't just have the `productId` (a small integer) but we have ALL the product columns (`productName`, etc.) that are **duplicated for each sale**! In termes of storage space it is huge.

## 2. Get familiar with BigQuery and its interface

### 2.1. First steps with the interface
You can access the Google Cloud Platform (GCP) Console at: https://console.cloud.google.com/. Then click on _Big Query_ and choose project `da-bootcamp-2023` on the top left-hand side corner of the page.

Or you can directly go to: https://console.cloud.google.com/bigquery?project=da-bootcamp-2023

You will have to log into a Google account. Use your personal one: if everything went as planned, you should have already been added to the `da-bootcamp-2023` project with your personal Gmail address. Otherwise, please raise your hand!

You should land up on a page similar to this one:

<img src="bigquery_homepage.png" width="800">

You may have noticed the `bigquery-public-data` line in the _Explorer_ block. It is a public _project_ published by Google, that contains multiple open datasets you can use. To make in also appear in your interface, click on _"+ Add"_ in the top bar, choose _"Public datasets"_ and click on one of them, for instance _"About Covid-19 public datasets"_ and then _"VIEW DATASET"_.

Alternatively, you can directly follow this link: https://console.cloud.google.com/marketplace/product/bigquery-public-datasets/covid19-public-data-program

### 2.2. The hierarchy

All the data in BigQuery is structured in a hierarchical way, that you can see in the _Explorer_ block:

1. **Project** (here `bigquery-public-data`),

2. **Dataset** (e.g. `covid19_nyt`),

3. **Table** (e.g. `mask_use_by_county`).

A table can be called by its full name: `project.dataset.table` (e.g. `bigquery-public-data.covid19_nyt.mask_use_by_county`).
In case you already selected the `bigquery-public-data` project, though, then you can skip the first part and just call it `covid19_nyt.mask_use_by_county`.


In this tutorial, we will mostly:
* Read data from the public dataset `bigquery-public-data.covid19_open_data`,
* Read and write data from and to our dataset `da-bootcamp-2023.myfirstname`.

### 2.3. Create my dataset

The first step for you is to create your personal dataset, where you will then create your tables.

To do so, click on the three dots and click on _"Create dataset"_:

<img src="bigquery_create_dataset.png" width="400">

Type your first name as the name of the dataset (lower case, no space, no special character), leave everything else as is and confirm. You should now see your personal dataset!

### 2.4. The first GoogleSQL query!

SQL is a specification: think of it as a family of query languages that are very similar. They share almost everything but each of them have some slight differences (more on this in section 3).

For now, you can consider that what you know from MySQL also applies to GoogleSQL (which is how Google calls the BigQuery flavor of SQL).

Let's make this query to test that everything works. It is silly and useless but free... (we will see later on that each action we make on BigQuery involves some charges). Maybe you don't know it, but you can make a `SELECT` without any table to return just a constant value:
```sql
SELECT 'hello' AS message
```

From the BigQuery homepage, click on _"COMPOSE A NEW QUERY"_ or the _"+"_ in the tab bar:

<img src="new_query.png" width="270">

Then, in the query editor, type your query and click on _"RUN"_ (you can also hit CTRL+Enter).

<img src="query_editor.png" width="800">

Several remarks about this screen:
* In the bottom part of the screen, you can see (and export) the results of your query.

* You can also have _"EXECUTION DETAILS"_, that gives you some measures about your job (time, consumption, etc.).
* Also, _"EXECUTION GRAPH"_ gives you interesting details about the interals of the query engine: it tells you the exact steps that have been done to get your query.

* In the top right corner, there is this message: `This query will process 0 B when run.` This gives you the cost of your query (very important!). More on this in section 3.

* On the top bar, you can notice the "SAVE" button: you can (should) use it!

For now, there is not much execution details nor execution graph, because the query is too simple. But there will be very soon!

## 4. How to optimize time and cost despite columnar storage?

### 4.1. LIMIT is useless

Let's try this query (**don't run it, just write it in the editor!**)
```sql
SELECT * FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

You should see in the top right corner a green check that says:
> This query will process 11.71 GB when run.
By the way, if instead of the green check you get a red cross, it will tell you the error in your query and where it comes from.

So let's change the query to save some money, because say we only need the first 100 rows to see what the data looks like:
```sql
SELECT * FROM `bigquery-public-data.covid19_open_data.covid19_open_data` LIMIT 100;
```

Easy, right?

Oops! The cost is the same...
> This query will process 11.71 GB when run.

Let's look at the table information (just open the `covid19_open_data` dataset and then on the `covid19_open_data` table inside).

<img src="table_info.png" width="600">

Actually, 11.71 GB corresponds to the whole table! But why read the whole table since we need only the first 100 rows.

The answer is simple: because BigQuery **stores data in columns** (it is a **columnar storage**) and not in rows.

So if you need only the first 100 rows of each column, it still has to read all the columns. And once a column is read, it does not matter how many rows we need: it takes roughly the same time (and exactly the same cost) to read 1 value or 10 million values.

---
If there is a single thing you should remember from BigQuery it is this:
* Data is stored in columns.
* Each column accessed is entirely billed (except for partitions, but we will see that later).
* Billing is per data read and written (not per data returned).
* Always look at the cost in the top right corner before running a query.
---

So what can we do to pay (and wait) less?

### 4.2. Think before doing

> This is the first lesson in the big data world: do less and think more before running a query.

A query can be costly and time consuming so we want to make sure it works, it is useful and it gives the right results before running it.

There are two (free) very interesting tools that we highly recommend you use extensively:

#### 4.2.1. Data schema

You can get the list of all columns along with their type.

<img src="table_schema.png" width="800">

#### 4.2.2. Data preview

> **This it THE tool you must use. It is free. Use it instead of `SELECT * FROM ... LIMIT ...`.**

<img src="table_preview.png" width="900">

### 4.3. SELECT fewer columns

Now that we have looked at the preview and we know what the data looks like, let's ask ourselves: do I really need all the columns?

Yes, because since BigQuery is a **columnar storage**, each column that we remove will reduce the cost and time by **a lot**!

Remember that in the big data world, datasets are **denormalized**. It means that all the columns anyone _might_ once need are all there, pre-joined for you. But it means there are *a lot* of columns.

We didn't find an easy way to get the total number of columns in a table (if you find one, we are interested!). But we still can get the information by querying the special metadata table called `INFORMATION_SCHEMA` (there is one per dataset):

```sql
SELECT
    COUNT(distinct column_name) 
FROM `bigquery-public-data.covid19_open_data.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = "covid19_open_data";
```

Anyway, the fact is: the dataset has 701 columns!! For sure, we don't need them all.

Let's try this one:
```sql
SELECT
    date,
    country_name,
    new_confirmed
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

Boom! Only **~500 Mb** of data processed: divided cost and time by almost 20!

Now we can run the query :)

In the bottom part of the window, you can see the results as a grid. You can also look at execution details to see how much time it took and how much data has been processed (and other details).

Also useful: on the very bottom bar, you have _"PERSONAL HISTORY"_ where you can access all the previous queries you made (even if you forgot to save them in the editor..)

<img src="execution_details_and_history.png" width="800">

## 5. Partitioning and clustering - how to save time and money?

This is quite annoying that we must query the whole column each time we need only part of it... Could we do something about it?

Well, there is no magical solution but it turns out that yes, we can do something: it is called *partitioning* and *clustering*.

First let us say that the word _clustering_ is really a poor choice here, since it is more some king of _sorting_ - but so it is. So what is it about?

### 7.1. Partitioning

The broad idea is to split all the columns in _chunks_ (**partitions**) according to the value of one of them.

Say we often access the COVID-19 data by date (i.e. with a `WHERE date ...` filter). Then it would be clever to:
1. Split the table into chunks, for instance one chunk per month (or multiple months in one chunk).
2. Write in some easily-accessible (e.g. in-memory) index the mapping between the month and the memory address where we stored this chunk.

This way, when we do a query like `SELECT ... WHERE date = "2021-05-15`, BigQuery can use the index to read (and charge for) **only** the **2021-05** chunk (which contains all columns for which the date is in May 2021).

Perfect, isn't it? Well... there is no free lunch.

* The biggest drawback: you can only **partition by one column** in a table. So you have to think hard in advance and choose the right one!

* It is costly to change the partitioning column (aka _repartition_).

* You can have **at most 4 000 partitions per table**.

* The partitioning index takes some additional space to store.

NB: All this means that it is a bad idea to partition on a column that has a lot of different value (not even speaking of a unique column like an ID - which would be a _very_ bad idea).

For more information about partitioning: https://cloud.google.com/bigquery/docs/partitioned-tables

### 5.2. Clustering

Again the word _clustering_ is inadequate (it is more _sorting_). So what does _clustering_ do?

First of all, as opposed to partitioning, clustering can be set on a list of columns (in a fixed order).

What BigQuery does it **sort the rows of the table**:
* By the first column,
* Then (if the values in the first column are equal) by the second column,
* Etc.

Why is it interesting? Well because there are clever lookup algorithms that work well when data is sorted (like for instance dictotomic search: https://en.wikipedia.org/wiki/Dichotomic_search).

It is not as good as partitioning but still:

* Within a single (large) partition, it helps reduce the amount of data processed.

* If you want to filter by the columns you clustered by (but not partitioned by) then clustering helps (not as good as if you partitioned by the filter you use, but better than nothing or than repartitioning the whole table).

### 5.3. Partitioning and clustering together in practice

If you look into the _"DETAILS"_ page of the `covid19_open_data` table you will see that it is not partionned nor clustered.

So we will create a new table with:
* Only the columns we need,
* Some nice partitioning and clustering.

```sql
CREATE TABLE `myfirstname.covid19_first`
PARTITION BY date
CLUSTER BY date,country_name
AS
SELECT
  date,
  country_name,
  new_tested,
  new_confirmed,
  new_hospitalized_patients,
  new_deceased
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

A couple of remarks:

* We can only partition on a `DATE` or numerical column (`INT64`, `FLOAT`, etc.). If you need to partition on a `STRING` column, you can use the very handy function `FARM_FINGERPRINT` that takes a string and returns an integer (more on this here: https://cloud.google.com/bigquery/docs/reference/standard-sql/hash_functions#farm_fingerprint).

* The number of partitions is determined automatically by BigQuery.

* It takes some time for the nuber of partitions to be updated (maybe a couple of minutes). You can go to the _"DETAILS"_ section of the new table and click _"REFRESH"_ until the partition number is not zero.

* Different dates can end up in the same partition (if there are more distinct dates than possible partitions).

* Why cluster by `date`, since we already partitioned by `date`? Well because of the revious remark. If you have multiple dates in the same partition, you may still want to optimize the query time _within_ the partition.

Here you see illutrated in practice the fact we talked about in the introduction, that with big data query engines you loose flexibility. You must plan in advance (when you create the table) for the type of queries you will do in the future...

### 5.4. Query a partitioned and clustered table

To test if you undestood these concepts, try to guess the amount of data processed by each of the following queries (not in absolute terms of course, but guess which one is more costly than which one):

```sql
-- Query A: no filter
SELECT * FROM `myfirstname.covid19_first`;

-- Query B: filter on one day
SELECT * FROM `myfirstname.covid19_first`
WHERE date = "2021-05-15";

-- Query C: filter on one month
SELECT * FROM `myfirstname.covid19_first`
WHERE date BETWEEN "2021-05-01" AND "2021-05-31";

-- Query D: filter on one country
SELECT * FROM `myfirstname.covid19_first`
WHERE country_name = 'France';

-- Query E: filter on one day and one country
SELECT * FROM `myfirstname.covid19_first`
WHERE
  date = "2021-05-15"
  AND country_name = 'France';

-- Query F: filter on one month and one country
SELECT * FROM `myfirstname.covid19_first`
WHERE
  date BETWEEN "2021-05-01" AND "2021-05-31"
  AND country_name = 'France';

```

And the answer is...

| Query                                    | Amount of data processed |
| ------------------------------------     | ------------------------ |
| A: no filter                             | 619.27 MB                |
| D: filter on one country                 | 619.27 MB                |
| C: filter on one month                   | 21.47 MB                 |
| F: filter on one month and one country   | 21.47 MB                 |
| B: filter on one day                     | 702.14 KB                |
| E: filter on one day and one country     | 802.14 KB                |

What do we see?

Partitioning works as expected:

* One month query processes 31 times more data than one day query.

* One month query takes ~30 times less time than the full dataset query (there is around 3 years of data in the table).

* Filtering by country has no particular effect, since the table is not partiioned by country.


But what is weird, is that clustering seems to have no effect at all... There are two reasons to that:

* **BigQuery cannot estimate in advance the cost reduction due to clustering**. So the estimate it gives you is a _worst case scenario_. You can only get the _actual_ amount of data processed if you _actually_ run the query.

* But in this case, even if we run the queries, we don't see a difference - for a tricky reason.<br/>
Remember clustering sorts by `date` and then **if (and only if!) the day is the same**, then it sorts by `country_name`. But the amount of data for one day is tiny (~800 KB). So the sort by `country_name` **within one day** is useless (800 KB is probably below the size of the smallest data chunk stored by BigQuery anyway).

So it is a bad idea to cluster by a column with has many distinct values - especially if it is the first clustering column.

### 5.5. See the effect of clustering

#### 5.5.1. First attempt
A better choice seems then to be to partition by `country_name` only.

```sql
CREATE TABLE `myfirstname.covid19_second`
PARTITION BY date
CLUSTER BY country_name
AS
SELECT
  date,
  country_name,
  new_tested,
  new_confirmed,
  new_hospitalized_patients,
  new_deceased
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

Now let's check:
```sql
SELECT * FROM `myfirstname.covid19_second`
WHERE country_name = 'France';
```

<img src="data_processed_bis.png" width="450">

Still no luck! It processes the whole dataset even when we ask for one country. So no effect of clustering at all...

What happened? Well now the issue is the interaction between clustering and partitioning. Indeed, remember that **clustering occurs only within each partition**. So even if we just cluster by `country_name` (which should work), the partitioning by `date` kills the potential effect of clustering. The partitions (by day) are so small that clustering has no effect.

#### 5.1.2. Second attempt
So what we need to do is to increase the size of the partitions. Instead of partitioning by day we will partition by year.

To do so, we will use the hendy function `DATE_TRUNC` which truncates the date (here to the year). For instance, a date like `2021-05-15` will be transformed into `2021-01-01` (like all other days of 2021). So all these 2021 days will end up in the same partition.

```sql
CREATE TABLE `myfirstname.covid19_third`
PARTITION BY DATE_TRUNC(date, YEAR)
CLUSTER BY country_name
AS
SELECT
  date,
  country_name,
  new_tested,
  new_confirmed,
  new_hospitalized_patients,
  new_deceased
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

Now let's check:
```sql
SELECT * FROM `myfirstname.covid19_third`
WHERE country_name = 'France';
```

<img src="data_processed_ter.png" width="400">

Hurrah! This time it worked. The amount of data processed is only 40 MB, which is much smaller than the 619 MB of the whole table.

NB: The _estimated_ amount displayed in the top right corner is still overestimated (619 MB), because BigQuery cannot know in advance how much it will save thanks to clustering.

So to recap on clustering:

---
* Don't cluster on a column that has too many distinct values.

* The order of the columns in clustering matters: it sorts by the first one first, then the second one, etc. If the first one has many distinct values, clustering by the second one becomes useless.

* Clustering is useless if you also partition into small partitions.

---

## 6. Group by

About group by, there is not much to say. No surprise here. You can for instance run:

```sql
SELECT
  date,
  SUM(new_confirmed) as total_new_confirmed
FROM `myfirstname.covid19_bis`
GROUP BY date
```

It will work as expected and will process exactly the same amount of data as:
```sql
SELECT
  date,
  new_confirmed
FROM `myfirstname.covid19_bis`;
```

Of course, if you add a `WHERE` clause, then you can benefit from partitioning and clustering, the same as we saw before.

## 7. Joins

### 7.1. First attempt

Let's create a `population` table with the population of each country per date. This would be useful to have this baseline to compute some statistics.

```sql
CREATE TABLE `myfirstname.population`
PARTITION BY date
CLUSTER BY country_name
AS
SELECT
  date,
  country_name,
  population
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

And make the `JOIN`:
```sql
SELECT * FROM `myfirstname.covid19_third` c
JOIN `myfirstname.population` p
  ON p.country_name = c.country_name
  AND p.date = c.date;
```

And... wait.

Well, the query is very slow: more that 13 minutes and it is still running. If you are patient enough, you will eventually see it fail with this error: `too many results`.

But wait: the number of result rows should be exactly the same as the number of rows in the original tables (since it is a one-to-one mapping)... What happened? Let's have a look at the _"EXECUTION GRAPH"_. It is important to look at it when a query takes too long or fails, to understand what is going on.

<img src="join_exploding.png" width="800">

You can see that there is an intermediate step with almost 50 billion rows that most probably is the culprit!

If you want, you can try and play with partitioning and clustering on both tables but sadly, it will not help here.

### 7.2. How to make the join work?

But wait, 22 million rows is quite a lot for the `population` table. If the data spans 3 years, it corresponds to 13 000 days. So it would mean there is ~1700 distinct countries. That is a little bit too much (there is around 200 countries in the world). So what happened?

Let's inspect the data closer:
```sql
SELECT *
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE date='2021-09-06'
AND country_name = 'Brazil';
```

There is ~5600 rows just for this one day this one country!

This is because we forgot about the `location_key` column (which is e.g. `BR_CE_231140` or `BR_ES_320500` etc.)!

We totally misunderstood the original dataset: one row is not one _country_ but one _location_ (maybe a city or district).

So let's recreate our tables but this time we include the `location_key` column and join on it.

NB: Of course we could ignore the location and do a `GROUP BY` country if we were interested only in the countries. But we keep the location for the demonstration, so that the dataset is not too small.

```sql
CREATE TABLE `myfirstname.covid19_loc`
AS
SELECT
  date,
  location_key,
  country_name,
  new_tested,
  new_confirmed,
  new_hospitalized_patients,
  new_deceased
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

```sql
CREATE TABLE `myfirstname.population_loc`
AS
SELECT
  date,
  country_name,
  location_key,
  population AS population
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`;
```

<img src="join_working_1.png" width="750">

Boom! The query completes in 19 seconds! Finally ✌️

### 7.3. Debrief

So what happened in the first case?

Recall that one day and one country could have up to 5K rows and we joined on day and country only.

So for each country and day, the `JOIN` creates 5K x 5K = 25 M rows! If we have 200 countries and 3 x 365 days, it is normal that the size of the final result is insane. 

> NB: With our mistake, we almost did a `CROSS JOIN` instead of an `INNER JOIN`.

So the take-home message for joins is:

---
* Beware of duplicates.

* Make sure you are joining on fields that are unique.

* Look at the execution graph to see what is happening.
---
<br/>

> A side note partitioning and clustering.
> 
> For joins they have no impact on the cost, because we have to read all the data anyway and only reading data is charged.
> 
> For large joins, they can help speed up a query though. In this case, there is no noticable impact (you can try multiple combinations and see by yourself).

## 7. Running BigQuery from Python

To have a unified processing pipeline, we would like to run our queries from Python, same as we would do with regular SQL (e.g. MySQL).

### 7.1. BigQuery setup in Python

#### 7.1.1. Install `gcloud` Command-Line Interface (CLI)
To set up the authentication, we need to first install `gcloud` CLI. It is a command that you will be able to run from the terminal to control GCP.

Follow the instructions that correspond to your operating system. You will find them here: https://cloud.google.com/sdk/docs/install

Don't forget to initialize `gcloud`:
```bash
gcloud init
``` 
Then follow the instructions to define:
* The default account you want to use (your personal Google account).
* The default project you want to use: `da-bootcamp-2023`.

#### 7.1.2. Set up Application Default Credentials (ADC)
To authorize connection from Python, we first need to generate special "application" credentials. It is something like a big password that will allow you to make queries from Python as if you were connected to your Google account.

You will find all the instructions to do so on this page: https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev

But in short, you just need to type this command on your terminal:
```bash
gcloud auth application-default login
```
And then, in the browser page that will open, you need to authenticate with your Google credentials.

You should see a message in the terminal that looks like:
```
Credentials saved to file: [/standard/config/path/application_default_credentials.json]
```
You can have a look at this file: it contains the credentials needed to connect to your Google Account with a command line. And it is also this file that the Python library will look at to authenticate you.

#### 7.1.3. Install the Python BigQuery library
As usual, simply run:
```bash
pip install google-cloud-bigquery
# OR
conda install google-cloud-bigquery
```

#### 7.1.4. Testing that everything works
To test that everything went fine, you can run the following Python code. It should automatically authenticate you and print `hello` which is the results of this (free) test query.

In [1]:
from google.cloud import bigquery

client = bigquery.Client(project="da-bootcamp-2023")
# For some reason, the library does not detect the default project previously set up with `gcloud init`
# so you have to specify it manually each time.

QUERY = "SELECT 'hello' AS message"
query_job = client.query(QUERY)
rows = query_job.result()

for row in rows:
    print(row.message)

hello


### 7.2. BigQuery setup with Pandas

The first thing to note is that you can already (with the previous code) do:

In [2]:
query_job.to_dataframe()

,message
0,hello


But BigQuery is even more nicely integrated with Pandas, with the function `pd.to_gbq` and `pd.read_gbq` which work similarly to `pd.to_sql` and `pd.read_sql`.

To use them, you need to install `pandas-gbq`:

```bash
pip install pandas-gbq
# OR
conda install pandas-gbq
```
Let's try!

In [3]:
import pandas as pd

df = pd.read_gbq("SELECT 'hello' AS message", project_id="da-bootcamp-2023")
df

,message
0,hello


In [4]:
# Replace "myfirstname" by your first name
df.to_gbq("myfirstname.hello", project_id="da-bootcamp-2023", if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 13706.88it/s]


In [5]:
pd.read_gbq("myfirstname.hello")

,message
0,hello


### 7.2. Running real queries from Python

Now it is time to run an actual query from Python!

But first, let's do a dry-run: it gives you the same (free of charge) cost estimate of your query that you would get from the console.

In [6]:
QUERY = "SELECT COUNT(*) FROM `myfirstname.covid19_third`"

query_job = client.query(QUERY, job_config=bigquery.QueryJobConfig(dry_run=True))
f"This query will process {query_job.total_bytes_processed} bytes."

'This query will process 0 bytes.'

In [10]:
df = pd.read_gbq(QUERY, project_id="da-bootcamp-2023")
df

,f0_
0,22756333


### 7.3. Executing other operations

Actually, you can also use the Python library to execute any action that you would execute through the console. It may be handy to make your code more reproducable and shareable (execute Python code instead of clicking on a Web interface).

For instance, you can automate the creation of a new dataset if it does not already exist:

In [8]:
from google.api_core.exceptions import NotFound

DATASET = "myfirstname"
try:
    client.get_dataset(DATASET)
    print("Dataset already exists: nothing to do.")
except NotFound:
    client.create_dataset(DATASET)
    print("Dataset created")
    pass

Dataset already exists: nothing to do.


You can also delete a table if you want:

In [9]:
client.delete_table("myfirstname.hello")

## 8. Quotas

There are two types of quotas in BigQuery:

### 8.1. System quotas
These are imposed by Google. You cannot change them except if you ask Google nicely. Some of them you cannot change because they are hard technical limits.

There are _a lot_ of them. One example of system quotas is the maximum of 4 000 partitions per table that we already spoke about. But there are many more.

Here is the full list: https://cloud.google.com/bigquery/quotas

### 8.2. Organization quotas
These are imposed by your account administrator, i.e. us, to make sure you do not run weird stuff that would incur crazy charges. 

You can see on [the _"Quotas"_ page](https://console.cloud.google.com/iam-admin/quotas?authuser=2&project=da-bootcamp-2023&pageState=(%22allQuotasTable%22:(%22s%22:%5B(%22i%22:%22serviceTitle%22,%22s%22:%220%22),(%22i%22:%22currentPercent%22,%22s%22:%221%22),(%22i%22:%22sevenDayPeakPercent%22,%22s%22:%220%22),(%22i%22:%22currentUsage%22,%22s%22:%221%22),(%22i%22:%22sevenDayPeakUsage%22,%22s%22:%220%22),(%22i%22:%22displayName%22,%22s%22:%220%22),(%22i%22:%22displayDimensions%22,%22s%22:%220%22)%5D,%22f%22:%22%255B%257B_22k_22_3A_22_22_2C_22t_22_3A10_2C_22v_22_3A_22_5C_22Query%2520usage_5C_22_22%257D%255D%22,%22r%22:200))) what your quota is and how much you already used:

<img src="quotas.png" width="950">


## 8. Take-home message: the main differences between MySQL et GoogleSQL

There is a _lot_ more to say about BigQuery but we let you practice on your own, now that you know pretty much all the essentials.

If you have questions, the best place to look at is BigQuery official documentation: https://cloud.google.com/bigquery/docs/

Here is what you should remember:

#### **There is no indexes and no foreign keys**
Nothing is indexed in BigQuery. Your only hope is to use partitioning and clustering to reduce time and cost.

#### **Joins are tricky and can be slow**
This is a consequence of the previous point and of the fact that the data can be spread across multiple locations.

Always make sure you are joining on fields that have **unique values**. If the query is too long, look at the **execution graph**.

#### **All fields are nullable**
In MySQL you specify for each field whether or not it can be null (which is important for indexes that cannot be nullable). In BigQuery, all fields can be null.

#### **Time overhead**
BigQuery is not good at responding fast to small simple queries. So small queries take longer than with a traditional relational database.

Broadly speaking, it is not made for real-time production queries ([OLTP](https://en.wikipedia.org/wiki/Online_transaction_processing)) but for _a posteriori_ analytics query ([OLAP](https://en.wikipedia.org/wiki/Online_analytical_processing)).

#### **Partitioning and clustering**
These features usually don't exist in that form in relational databases like MySQL. You should understand how they work (see above).

#### **Pay as you go**
This is due to the _cloud_ nature of the service. You pay for _every_ query based on how much data has been _initially read_ and _finally written_ by your query.

> The current costs are $6.25 per TiB (the first TiB per month per account is free)
>
> More on this: https://cloud.google.com/bigquery/pricing#analysis_pricing_models

Note that for now (but this may change in the future):

* BigQuery does not charge based on query time.
* It does not charge intermediate reads and writes that it makes under the hood (only data that you explicitly read or write).
* If you run a query multiple times in a row, only the first time gets charged, thanks to a _caching_ mechanism (data is stored in some cheaper-to-access storage).


So you need to think twice before running a query, and especially look at the estimated execution costs.

#### **Closed-source engine**
BigQuery is a proprietary technology. In some way, it is similar to the open-source engine [Apache Cassandra](https://en.wikipedia.org/wiki/Apache_Cassandra) (which is also columnar and has similar partitioning and clustering logic). But we don't know exactly what the engine behind BigQuery is.